In [ ]:
!pip install -q gradio
!pip install huggingsound

In [2]:
# cargamos el modelo de huggingsound
import re
from huggingsound import SpeechRecognitionModel

# creación del modelo
model = SpeechRecognitionModel("patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm")

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


05/01/2022 00:58:50 - INFO - huggingsound.speech_recognition.model - Loading model...


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [3]:
def sim_jac(s1, s2):

    bigrams_s1 = []
    bigrams_s2 = []

    for i in range(len(s1) - 1):
        bigrams_s1.append(s1[i:i+2])
    
    for i in range(len(s2) - 1):
        bigrams_s2.append(s2[i:i+2])

    c_common = 0

    for i in bigrams_s1:
        if bigrams_s2.count(i) > 0:
            c_common += 1

    return c_common / ((len(s1) - 1) + (len(s2) - 1) - c_common)

In [33]:
def plantilla_funcion(instruccion):

    before_keyword, keyword, after_keyword = instruccion.partition('nombre')

    recomendacion = ''
    output = ''

    if len(after_keyword) == 0:
        recomendacion = f'¡No me dijiste el nombre de la función!'

    else:
        # quitar acentos
        name_func = after_keyword.split(' ')[1]

        if instruccion.strip().split(' ')[-1] == name_func:
            parametros = ''

        else:
            before_keyword, keyword, after_keyword = instruccion.partition('parámetros')

            if len(after_keyword) == 0:
                parametros = ''
                recomendacion = f'¡No me dijiste el nombre de los parámetros!'

            else:
                parametros = after_keyword.split(' ')[1:]
                parametros = ', '.join(parametros)

        output = f"""
        def {name_func}({parametros}):
            
        """

    return output, recomendacion

In [34]:
plantilla_funcion('definir función con nombre magica y parámetros')

('\n        def magica():\n            \n        ',
 '¡No me dijiste el nombre de los parámetros!')

In [6]:
cadena = '    holaa   mundo    '
cadena.strip()

'holaa   mundo'

definir función genérica con parametros x g s

In [7]:
def main(instruccion):

    # lista de comandos disponibles
    comandos = ['definir función',
                'definir parámetro',
                'definir variable']

    # plantillas
    plantillas = [plantilla_funcion]

    # buscar el comando correspondiente más cercano al deseado
    scores = []
    ind_max_score = 0

    # similitud de Jaccard para las primeras dos palabras de la grabación
    for i, comando in enumerate(comandos):
        scores.append(sim_jac(instruccion.split(' ')[0],
                              comando.split(' ')[0]) * 0.3 + sim_jac(instruccion.split(' ')[1],
                                                                     comando.split(' ')[1]) * 0.7)

        # guardamos índice de la instrucción con más similitud
        if scores[-1] > scores[ind_max_score]:
            ind_max_score = i
    
    return plantillas[ind_max_score](instruccion)

In [35]:
import gradio as gr

def transcribe(audio, state=""):    
    transcriptions_es = model.transcribe([audio])[0]    
    # Algoritmo here
    codigo, recomendacion = main(transcriptions_es['transcription'])
    return codigo, recomendacion

inputs = gr.inputs.Audio(label="Dar click para escuchar tu voz", type="filepath", source="microphone")
output1 = gr.outputs.Textbox(label="Asi se ve tu código")
output2 = gr.outputs.Textbox(label="Sugerencia")

title = "Expresate con voz"
description = "Aplicación que ayuda a programar a traves de tu voz"
examples = ['definir función', 'definir variable', 'definir clase']
article = "<a  style='color:#eb9f59;' href = 'https://github.com/gandres-dev/Hackaton-Common-Voice'> Repositorio de la app"
demo = gr.Interface(fn=transcribe, inputs=inputs, outputs=[output1,output2],
                    title=title, description=description, article=article,
                    allow_flagging="never", theme="darkpeach", examples=examples,
                    #live=True
                    )

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:317: UserWarning: 'darkpeach' theme name is deprecated, using dark-peach instead.
  f"'{theme}' theme name is deprecated, using {DEPRECATED_THEME_MAP[theme]} instead."


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
Running on public URL: https://17937.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


In [9]:
!pip install ipywebrtc

In [10]:
from google.colab import output
output.enable_custom_widget_manager()

Crea el widget de audio

In [11]:
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Reescribir audio ahora en formato .wav

In [21]:
# Escribe el formato en webm
with open('test_audio.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i test_audio.webm -ac 1 -f wav test_audio.wav -y -hide_banner -loglevel panic

## Modelos

In [22]:
audio_paths = ["test_audio.wav"]

# Obtenemos la frase del usuario
transcriptions_es = model.transcribe(audio_paths)[0]
# Obtener la frase
transcriptions_es['transcription']

100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


'definir mágica'

In [23]:
main(transcriptions_es['transcription'])

UnboundLocalError: ignored

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>